In [42]:
import pickle
import numpy as np
from itertools import combinations

from scipy.stats import wilcoxon
from statsmodels.stats.multitest import multipletests

In [22]:
def extract_lower_triangular(matrix):
    return matrix[np.tril_indices(matrix.shape[0], k=-1)]

def extract_stage_lower_triangular(connectivity, stage):
    return extract_lower_triangular(connectivity[stage].get_data("dense").squeeze())

In [2]:
# load data
with open('/Users/soroush/Documents/Code/freelance-project/vielight/vielight_close_loop/dpli_connectivity.pkl', 'rb') as f:
    dpli_connectivity = pickle.load(f)

with open('/Users/soroush/Documents/Code/freelance-project/vielight/vielight_close_loop/wpli_connectivity.pkl', 'rb') as f:
    wpli_connectivity = pickle.load(f)

In [29]:
flatten_connectivity = dpli_connectivity["pre"].get_data()
connectivity = dpli_connectivity["pre"].get_data("dense").squeeze()

In [52]:
triangular_matrice = {stage: extract_stage_lower_triangular(wpli_connectivity, stage) for stage in ["pre", "during", "post"]}

In [53]:
from itertools import combinations

# calculate the correlation between the lower triangular matrices
correlations = {}
for stage1, stage2 in combinations(triangular_matrice.keys(), 2):
    correlations[f"{stage1}_{stage2}"] = wilcoxon(triangular_matrice[stage1], triangular_matrice[stage2])

In [54]:
correlations

{'pre_during': WilcoxonResult(statistic=18831.0, pvalue=1.3616590055863041e-173),
 'pre_post': WilcoxonResult(statistic=41988.0, pvalue=3.6044230110910713e-150),
 'during_post': WilcoxonResult(statistic=110922.0, pvalue=1.382048093713617e-90)}

In [55]:
# Combine p-values for correction
p_values = list(map(lambda x: x[1],correlations.values()))

# Apply Bonferroni correction
corrected_p_values = multipletests(p_values, alpha=0.05, method='bonferroni')

In [56]:
p_values

[1.3616590055863041e-173, 3.6044230110910713e-150, 1.382048093713617e-90]

In [57]:
corrected_p_values

(array([ True,  True,  True]),
 array([4.08497702e-173, 1.08132690e-149, 4.14614428e-090]),
 0.016952427508441503,
 0.016666666666666666)